In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests

In [2]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
    
use_selftest=False ## speedup

TensorFlow Version: 1.2.1


C:\Users\Andreas\AppData\Local\conda\conda\envs\dl\lib\site-packages\ipykernel\__main__.py:7: UserWarning: No GPU found. Please use a GPU to train your neural network.


https://www.tensorflow.org/api_docs/python/tf/saved_model/loader/load
https://www.tensorflow.org/api_docs/python/tf/get_default_graph

In [3]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    
    graph = tf.get_default_graph()
    image_input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob  = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return image_input, keep_prob, layer3_out, layer4_out, layer7_out

if use_selftest:
    tests.test_load_vgg(load_vgg, tf)

https://people.eecs.berkeley.edu/~jonlong/long_shelhamer_fcn.pdf

https://www.tensorflow.org/api_docs/python/tf/layers/conv2d

https://www.tensorflow.org/api_docs/python/tf/layers/conv2d_transpose

https://www.tensorflow.org/api_docs/python/tf/contrib/layers/l2_regularizer

AttributeError: module 'pandas' has no attribute 'computation'

pip install dask --upgrade
 

In [4]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    
    # conv2d
    padding = 'same'
    kernel_regularizer = tf.contrib.layers.l2_regularizer(0.001)
    kernel_size = 1
    
    vgg_layer7_conv2d = tf.layers.conv2d(vgg_layer7_out,num_classes,kernel_size,padding=padding,kernel_regularizer=kernel_regularizer)
    vgg_layer4_conv2d = tf.layers.conv2d(vgg_layer4_out,num_classes,kernel_size,padding=padding,kernel_regularizer=kernel_regularizer)
    vgg_layer3_conv2d = tf.layers.conv2d(vgg_layer3_out,num_classes,kernel_size,padding=padding,kernel_regularizer=kernel_regularizer)
    
    # conv2d_transpose     # add skip connections
    padding = 'same'
    kernel_size = 4
    strides=(2, 2)
    
    fcn_decoder_layer1a = tf.layers.conv2d_transpose(vgg_layer7_conv2d,num_classes,kernel_size,strides,padding,kernel_regularizer=kernel_regularizer)
    fcn_decoder_layer1b = tf.add(fcn_decoder_layer1a, vgg_layer4_conv2d)
    fcn_decoder_layer2a = tf.layers.conv2d_transpose(vgg_layer4_conv2d,num_classes,kernel_size,strides,padding,kernel_regularizer=kernel_regularizer)
    fcn_decoder_layer2b = tf.add(fcn_decoder_layer2a, vgg_layer3_conv2d)
    strides=(8, 8)
    fcn_decoder_layer3a = tf.layers.conv2d_transpose(vgg_layer3_conv2d,num_classes,kernel_size,strides,padding,kernel_regularizer=kernel_regularizer)


    
    # TODO: Implement function
    return fcn_decoder_layer3a

if use_selftest:
    tests.test_layers(layers)

https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits

In [5]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    labels = tf.reshape(correct_label, (-1, num_classes))
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_loss)

    return logits, train_op, cross_entropy_loss

if use_selftest:
    tests.test_optimize(optimize)

https://www.tensorflow.org/api_docs/python/tf/Session#run

In [6]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        print("next epoch")
        for image,label in get_batches_fn(batch_size):
            loss, _ = sess.run([cross_entropy_loss, train_op],feed_dict={input_image: image, correct_label: label, keep_prob: 1})
            print("Loss : {:.5f}".format(loss))
    pass

if use_selftest:
    tests.test_train_nn(train_nn)

In [ ]:
def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        correct_label = tf.placeholder(tf.float32, [None, None, None, num_classes])
        learning_rate = 0.000001#tf.placeholder(tf.float32, name='learning_rate')
    
        input_image, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess,vgg_path)
        nn_last_layer = layers(layer3_out, layer4_out, layer7_out, num_classes)
        logits, train_op, cross_entropy_loss = optimize(nn_last_layer, correct_label, learning_rate, num_classes)

        # TODO: Train NN using the train_nn function
        epochs = 11
        
        for epoch in range(epochs):
            batch_size = epoch+1
            train_nn(sess, 1, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image, correct_label, keep_prob, learning_rate)

        # TODO: Save inference data using helper.save_inference_samples
        #  helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)
            helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)

        # OPTIONAL: Apply the trained model to a video


if __name__ == '__main__':
    run()

Tests Passed
INFO:tensorflow:Restoring parameters from b'./data\\vgg\\variables\\variables'
next epoch
Loss : 30.49392
Loss : 35.26026
Loss : 27.74233
Loss : 35.12626
Loss : 26.07379
Loss : 32.21667
Loss : 40.48513
Loss : 26.51129
Loss : 33.77902
Loss : 30.55238
Loss : 27.31537
Loss : 37.10611
Loss : 35.53797
Loss : 35.28535
Loss : 32.05166
Loss : 32.84652
Loss : 32.23930
Loss : 36.93237
Loss : 34.56035
Loss : 30.59001
Loss : 32.95424
Loss : 30.67621
Loss : 29.98507
Loss : 33.67423
Loss : 32.37423
Loss : 34.41356
Loss : 39.18463
Loss : 39.71766
Loss : 33.76910
Loss : 46.53571
Loss : 41.23667
Loss : 33.17650
Loss : 40.77170
Loss : 39.06192
Loss : 37.01707
Loss : 30.28824
Loss : 39.07106
Loss : 44.45984
Loss : 31.25797
Loss : 38.15129
Loss : 40.04258
Loss : 44.51507
Loss : 39.91955
Loss : 42.89638
Loss : 43.64374
Loss : 34.37183
Loss : 41.89130
Loss : 36.51896
Loss : 38.18792
Loss : 42.49823
Loss : 49.35035
Loss : 39.46755
Loss : 42.39024
Loss : 39.92202
Loss : 42.76218
Loss : 44.80397
L